In [ ]:

# -------------------------------
# TAB 4: NLP EXTRACTION
# -------------------------------
with tabs[3]:
    st.header("🧠 NLP Extraction — Entity & Relation (Milestone 2)")

    if "logged_in" not in st.session_state:
        st.warning("⚠️ Please login first.")
    else:
        def extract_triples_from_text(text: str):
            """Extract minimal triples (S, P, O) even for single-sentence text."""
            try:
                nlp_local = spacy.load("en_core_web_sm")
            except Exception:
                st.error("spaCy model not found. Run: !python -m spacy download en_core_web_sm")
                return []
            doc = nlp_local(text)
            triples = []
            for sent in doc.sents:
                root = next((t for t in sent if t.dep_ == "ROOT"), None)
                if not root:
                    continue
                subj = next((w for w in root.lefts if "subj" in w.dep_), None)
                obj = next((w for w in root.rights if "obj" in w.dep_ or w.dep_ == "pobj"), None)
                if subj and obj:
                    triples.append((subj.text, root.lemma_, obj.text))
            if not triples and len(doc.ents) >= 2:
                triples = [(doc.ents[0].text, "related_to", doc.ents[1].text)]
            if not triples and len(doc) >= 3:
                tokens = [t.text for t in doc if not t.is_space]
                triples = [(tokens[0], tokens[1], tokens[2])]
            return triples

        triples_out = st.session_state.get("triples", None)

        if triples_out:
            st.success("✅ Automatically extracted triples from uploaded CSV!")
            st.dataframe(pd.DataFrame(triples_out, columns=["Subject","Relation","Object"]))
        else:
            st.info("No triples found in dataset. You can enter text below to extract manually:")
            text_input = st.text_area("Enter text for extraction", height=150)
            if st.button("🚀 Run NLP Extraction"):
                if not text_input.strip():
                    st.error("Please enter some text.")
                else:
                    triples_out = extract_triples_from_text(text_input)
                    st.session_state["triples"] = triples_out
                    st.dataframe(pd.DataFrame(triples_out, columns=["Subject","Relation","Object"]))
                    st.success("✅ Extraction completed and saved for Knowledge Graph visualization.")

